# 03 Bundle Playground

**Purpose**: Interactive exploration of bundles - paste YAML, see mount plan, experiment.

This is a PLAYGROUND, not a tutorial. For learning, see:
- `examples/01_load_and_inspect.py` - How to load bundles
- `docs/CONCEPTS.md` - What bundles are and why

---

In [ ]:
# Setup - run this cell first
import json
from pathlib import Path

import yaml
from amplifier_foundation import Bundle
from amplifier_foundation import load_bundle

## Option 1: Paste Your Bundle YAML

Edit the YAML below and run the cell to see the resulting Bundle object.

In [ ]:
# Edit this YAML - your bundle definition
bundle_yaml = """
bundle:
  name: my-test-bundle
  version: 1.0.0
  description: Testing bundle composition

session:
  orchestrator:
    module: loop-streaming
  context:
    module: context-simple
    config:
      max_tokens: 100000

providers:
  - module: provider-anthropic
    config:
      default_model: claude-sonnet-4-5

tools:
  - module: tool-filesystem
  - module: tool-bash

instruction: |
  You are a helpful assistant.
"""

# Parse and create Bundle
data = yaml.safe_load(bundle_yaml)
if not isinstance(data, dict):
    raise ValueError("Bundle YAML must parse to a dict")
bundle = Bundle.from_dict(data)

print(f"Bundle: {bundle.name} v{bundle.version}")
print(f"Description: {bundle.description}")

## Option 2: Load from File Path

Load an existing bundle from disk.

In [ ]:
# Load from path (edit path as needed)
# In Jupyter, you can use 'await' directly at the top level

# Default to minimal foundation bundle to avoid circular refs
foundation_path = Path("..") / "bundles" / "minimal.yaml"
bundle = await load_bundle(str(foundation_path))

print(f"Loaded: {bundle.name} v{bundle.version}")

## Inspect the Mount Plan

The mount plan is what `AmplifierSession.create()` uses.

In [ ]:
# Get mount plan
mount_plan = bundle.to_mount_plan()

# Pretty print
print(json.dumps(mount_plan, indent=2, default=str))

## Inspect Individual Sections

In [ ]:
# Session configuration
print("SESSION:")
print(json.dumps(mount_plan.get("session", {}), indent=2))

In [ ]:
# Providers
print(f"PROVIDERS ({len(mount_plan.get('providers', []))})")
for p in mount_plan.get("providers", []):
    print(f"  - {p.get('module')}: {p.get('config', {})}")

In [ ]:
# Tools
print(f"TOOLS ({len(mount_plan.get('tools', []))})")
for t in mount_plan.get("tools", []):
    print(f"  - {t.get('module')}")

In [ ]:
# Hooks
print(f"HOOKS ({len(mount_plan.get('hooks', []))})")
for h in mount_plan.get("hooks", []):
    print(f"  - {h.get('module')}")

In [ ]:
# Agents
agents = mount_plan.get("agents", {})
print(f"AGENTS ({len(agents)})")
for name in sorted(agents.keys()):
    print(f"  - {name}")

## System Instruction

In [ ]:
# Get system instruction
instruction = bundle.get_system_instruction()
if instruction:
    print(f"Length: {len(instruction)} chars")
    print("Preview (first 500 chars):")
    print(instruction[:500])
else:
    print("No instruction defined")

---

## Experiment!

Try changing the YAML above and re-running cells to see how the mount plan changes.

Ideas to try:
- Add more tools
- Change the provider model
- Add hooks
- Modify session config